#Лабораторная работа №5
##по теме «Методы решения обыкновенных дифференциальных уравнений»
##Васильев В. Вариант 8

##Задание:

*   Найти аналитическое решение ДУ: $y' = (x-1)^2 y^2$;
*   Создать в сценарии функцию для вычисления  значений полученного решения;
*   Создать в сценарии функцию для вычисления  значений численного решения ДУ методом Эйлера;



In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [ ]:
# Given data
a = 0
b = 1.5
h = 0.5

##Аналитическое решение

$$y'=\left({x-1}\right)^{2}\,{y}^{2}\;\to\;y=-\dfrac{3}{{x}^{3}-3\,{x}^{2}+3\,x+C-1}$$

При $x_0 = 0, \, y_0 = -1$ :
$$y=-\dfrac{3}{{x}^{3}-3\,{x}^{2}+3\,x+3}$$

In [ ]:
def exact_f(x):
  return -3 / (x**3 - 3 * x**2 + 3 * x + 3)

In [ ]:
data = []
for x_i in np.arange(a, b + h, h):
  data.append((x_i, exact_f(x_i)))
  # x.append(x_i)
  # y.append(exact_f(x_i))

exact_df = pd.DataFrame(data=data, columns=['x', 'y(x)'])
exact_df

,x,y(x)
0,0.0,-1.000000
1,0.5,-0.774194
2,1.0,-0.750000
3,1.5,-0.727273


##Метод Эйлера
$$y_{i+1} = y_i + h * f(x_i, y_i)$$

In [ ]:
def f(x, y):
  return (x - 1)**2 * y**2

In [ ]:
x = np.arange(a, b + h, h)
y = [-1]

for i in range(3):

  y_cur = round(y[i] + h * f(x[i], y[i]), 5)

  print(i, y_cur, x[i], y[i], f(x[i], y[i]), sep='\t'*2)
  y.append(y_cur)

y

0		-0.5		0.0		-1		1.0
1		-0.46875		0.5		-0.5		0.0625
2		-0.46875		1.0		-0.46875		0.0


[-1, -0.5, -0.46875, -0.46875]

In [ ]:
euler_df = pd.DataFrame({'x': x, 'y(x)': y})
euler_df

,x,y(x)
0,0.0,-1.00000
1,0.5,-0.50000
2,1.0,-0.46875
3,1.5,-0.46875


In [ ]:
error_euler = exact_df[['x']]
error_euler['E'] = abs(exact_df['y(x)'] - euler_df['y(x)'])
error_euler

,x,E
0,0.0,0.000000
1,0.5,0.274194
2,1.0,0.281250
3,1.5,0.258523


###Поясним метод эйлера графически




In [ ]:
def euler(a, b, h, y1):
  x = np.arange(a, b + h, h)
  y = [y1]
  for i in range(int((b-a)/h)):

    y_cur = round(y[i] + h * f(x[i], y[i]), 5)

    # print(i, y_cur, x[i], y[i], f(x[i], y[i]), sep='\t'*2)
    y.append(y_cur)
  return x, y

In [ ]:
x_plot = np.linspace(a, b, 100)
y_plot = exact_f(x_plot)

###Метод Эйлера на отрезке [a; b]

In [ ]:
h = 0.5
fig = go.Figure()

# fig.add_trace(go.Scatter(x=[a,b],
#                          y=[0, 0],
#                          title = 'Ось Х'))

fig.add_trace(go.Scatter(x=x_plot,
                         y=y_plot,
                         name='Точная функция'))
for i in range(5):
  h1 = h / 2**(i)
  x_plot1, y_plot1 = euler(a, b, h1, -1)

  fig.add_trace(go.Scatter(x=x_plot1,
                           y=y_plot1,
                           name=f'Mетод Эйлера с шагом h/{2**(i)}',
                           mode='lines+markers'))


fig

###Метод Эйлера, где количество точек меняется на каждом шаге

In [ ]:
h = 0.5
fig = go.Figure()


fig.add_trace(go.Scatter(x=x_plot,
                         y=y_plot,
                         name='Точная функция'))

x = np.arange(a, b, h)
t = [-1]
for i in x:
  h = 0.5
  x1 = i
  x2 = i + h

  x_euler1, y_euler1 = euler(x1, x2, h, t[-1])
  x_euler2, y_euler2 = euler(x1, x2, h/2, t[-1])


  # print(x1, x2, t, '!!!!!!!!!!!!!!!!',  x_euler1, y_euler1)
  j = 0
  while abs(y_euler1[-1] - y_euler2[-1]) > 1e-2:
    h /= 2
    j += 1
    x_euler1, y_euler1 = euler(x1, x2, h, t[-1])
    x_euler2, y_euler2 = euler(x1, x2, h/2, t[-1])

    # print(h, y_euler1[-1], y_euler2[-1], y_euler1[-1] - y_euler2[-1])

  t.append(y_euler1[-1])

  # print(x_euler2, y_euler2)

  fig.add_trace(go.Scatter(x=x_euler1,
                           y=y_euler1,
                           name=f'Mетод Эйлера с шагом h/{2**(j)}',
                           mode='lines+markers'))

  fig.add_trace(go.Scatter(x=x_euler2,
                           y=y_euler2,
                           name=f'Mетод Эйлера с шагом h/{2**(j+1)}',
                           mode='lines+markers'))


fig

## Метод Рунге-Кутта 4-го порядка
$$y_{i+1} = y_i * \frac{h}{6} (k_1 + 2\,k_2 + 2\,k_3 + k_4)$$
$$k_1 = f(x_i, \, y_i)$$
$$k_2 = f(x_i + \frac{h}{2}, \, y_i + \frac{h\,k_1}{2})$$
$$k_3 = f(x_i + \frac{h}{2}, \, y_i + \frac{h\,k_2}{2})$$
$$k_4 = f(x_i + h, y_i + h\,k_3)$$

In [ ]:
a = 0
b = 1.5
h = 0.5

In [ ]:
def runge_kutta(f, x0, y0, h):

    y = {0: -1}
    x = np.arange(a, b + h, h)

    for i in range(len(x)-1):
        k1 = f(x[i], y[x[i]])
        k2 = f(x[i] + h/2, y[x[i]] + (k1 * h)/2)
        k3 = f(x[i] + h/2, y[x[i]] + (k2 * h)/2)
        k4 = f(x[i] + h, y[x[i]] + h * k3)

        y_cur = y[x[i]] + (h/6) * (k1 + 2*k2 + 2*k3 + k4)
        y[x[i]+h] = y_cur

    return y

def runge_double_prosch(f, x0, y0, h):
  m1 = runge_kutta(f,x0,y0,h)
  m2 = runge_kutta(f,x0,y0,h/2)

  error = dict()
  for i in np.arange(a, b + 0.5, 0.5):
    error[i] = (abs(m1[i] - m2[i]))

  return error

In [ ]:
m1 = runge_kutta(f, 0, -1, h)
m2 = runge_kutta(f, 0, -1, h/2)

for i in np.arange(a, b + h, h):
  print(i, m1[i], m2[i], abs(m1[i] - m2[i]))

0.0 -1 -1 0
0.5 -0.7723468365923265 -0.7740950880753644 0.0017482514830379214
1.0 -0.7482123706643874 -0.7499053094605451 0.001692938796157728
1.5 -0.725548653741051 -0.7271819981664578 0.0016333444254067464


In [ ]:
a = 0
b = 1.5
h = 0.5
i = 0
y = {i: exact_f(i) for i in np.arange(a, b + 0.5, 0.5)}
while not(all([abs(runge_kutta(f,0,-1,h)[i] - exact_f(i)) < 1e-4 for i in np.arange(a, b + 0.5, 0.5)])):
  i += 1
  h /= 2

print(i, runge_kutta(f,0,-1,h), y)

for i in m1.keys():
  print(i, runge_kutta(f,0,-1,h)[i], y[i], abs(runge_kutta(f,0,-1,h)[i] - y[i]), sep='\t')

1 {0: -1, 0.25: -0.8383248127511764, 0.5: -0.7740950880753644, 0.75: -0.7528464783576242, 1.0: -0.7499053094605451, 1.25: -0.7469870472781146, 1.5: -0.7271819981664578} {0.0: -1.0, 0.5: -0.7741935483870968, 1.0: -0.75, 1.5: -0.7272727272727273}
0	-1	-1.0	0.0
0.5	-0.7740950880753644	-0.7741935483870968	9.846031173232905e-05
1.0	-0.7499053094605451	-0.75	9.469053945487804e-05
1.5	-0.7271819981664578	-0.7272727272727273	9.072910626950481e-05


In [ ]:
h = 0.25
temp = [runge_kutta(f,0,-1,h)[i] - exact_f(i) < 1e-4 for i in np.arange(a, b + 0.5, 0.5)]
print(runge_kutta(f,0,-1,h), temp, not(all(temp)))

{0: -1, 0.25: -0.8383248127511764, 0.5: -0.7740950880753644, 0.75: -0.7528464783576242, 1.0: -0.7499053094605451, 1.25: -0.7469870472781146, 1.5: -0.7271819981664578} [True, True, True, True] False


In [ ]:
runge = runge_kutta(f,0,-1,h)
runge_df = pd.DataFrame({'x': np.arange(a, b + 0.5, 0.5), 'y(x)': [runge_kutta(f,0,-1,h)[i] for i in np.arange(a, b + 0.5, 0.5)]})
runge_df

,x,y(x)
0,0.0,-1.000000
1,0.5,-0.774095
2,1.0,-0.749905
3,1.5,-0.727182


In [ ]:
runge_error = exact_df[['x']]
runge_error['R'] = abs(exact_df['y(x)'] - runge_df['y(x)'])
runge_error

,x,R
0,0.0,0.000000
1,0.5,0.000098
2,1.0,0.000095
3,1.5,0.000091


In [ ]:
total_df = exact_df
total_df['Euler_y'] = euler_df['y(x)']
total_df['Euler_error'] = error_euler['E']
total_df['Runge_y'] = runge_df['y(x)']
total_df['Runge_error'] = runge_error['R']


In [ ]:
total_df

,x,y(x),Euler_y,Euler_error,Runge_y,Runge_error
0,0.0,-1.000000,-1.00000,0.000000,-1.000000,0.000000
1,0.5,-0.774194,-0.50000,0.274194,-0.774095,0.000098
2,1.0,-0.750000,-0.46875,0.281250,-0.749905,0.000095
3,1.5,-0.727273,-0.46875,0.258523,-0.727182,0.000091


С точностью

In [ ]:
# h = 0.5
# x = [0]
# y = [10]
exps = range(1, 8)
iters = []
for exp in exps:
  E = 10**(-exp)
  h = 0.5
  i = 0
  while any(runge_double_prosch(f, 0, -1, h)[i] > E for i in np.arange(a, b + 0.5, 0.5)):
    i += 1
    h /= 2
    # print(E, i, h)

  iters.append(i)

iters

[0, 0, 1, 1, 2, 3, 4]

In [ ]:
# runge_df = pd.DataFrame({'x': x, 'y(x)': y})
# exact_df = pd.DataFrame(data=exact_y, columns=['x', 'y(x)'])
# # exact_df

In [ ]:
# runge_df

In [ ]:
# error = exact_df[['x']]
# error['E'] = exact_df['y(x)'] - runge_df['y(x)']
# error

In [ ]:
iters

[0, 0, 1, 1, 2, 3, 4]

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(exps),
                         y=iters,
                         mode='markers',
                         marker=dict(color='LightSkyBlue', size=15, line=dict(color='MediumPurple', width=3))))

fig.update_layout(title="Зависимость итераций от погрешности",
                  xaxis_title="Значение погрешности",
                  yaxis_title="Итерации")